# ADB 접속 정보

## 웹 db 접속 링크 
- https://g55136018110880-vs.adb.us-chicago-1.oraclecloudapps.com/ords/saltware/_sdw/
- saltware / Oracle_123456

## oracle 26ai를 PYTHON으로 접속에 필요한 사항
1) Wallet 및  Wallet Password
2) Wallet Directory
3) User ID
4) User Password
5) TNS Alias


In [78]:
ORACLE_DB_USER = "saltware"
ORACLE_DB_PASSWORD = "Oracle_123456"
ORACLE_DB_DSN = "vs_medium" 
ORACLE_WALLET_DIR = "/home/opc/wallet"
ORACLE_WALLET_PASSWORD = "Oracle_123456"

## DB 사용자 생성 (관리자 계정에서 실행)
```
-------------------------------------------------------------------
-- 사전준비
-- admin user에서 실행
-------------------------------------------------------------------
-- Create Database User and Initial Grants
create user saltware identified by "Oracle_123456";
grant dwrole to saltware;
grant unlimited tablespace to saltware;
grant create job to saltware;

-- Grant Database User Access to DBMS Packages
GRANT EXECUTE on DBMS_CLOUD_AI_AGENT to saltware;
GRANT EXECUTE on DBMS_CLOUD_AI to saltware;
GRANT EXECUTE on DBMS_NETWORK_ACL_ADMIN to saltware;
GRANT EXECUTE on DBMS_CLOUD to saltware;

BEGIN
    DBMS_CLOUD_ADMIN.ENABLE_PRINCIPAL_AUTH(
        provider => 'OCI',
        username => 'saltware'
    );
END;

-- ADD ROLES
GRANT CONNECT TO SALTWARE;
GRANT RESOURCE TO SALTWARE;
ALTER USER SALTWARE DEFAULT ROLE DWROLE,CONNECT,RESOURCE;


-- REST ENABLE
BEGIN
    ORDS_ADMIN.ENABLE_SCHEMA(
        p_enabled => TRUE,
        p_schema => 'SALTWARE',
        p_url_mapping_type => 'BASE_PATH',
        p_url_mapping_pattern => 'saltware',
        p_auto_rest_auth=> FALSE
    );
    -- ENABLE DATA SHARING
    C##ADP$SERVICE.DBMS_SHARE.ENABLE_SCHEMA(
            SCHEMA_NAME => 'SALTWARE',
            ENABLED => TRUE
    );
    commit;
END;
/

-- QUOTA
ALTER USER SALTWARE QUOTA UNLIMITED ON DATA;

```

In [79]:
import oracledb
import logging

log = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # 콘솔 출력
    ]
)

log.setLevel(logging.INFO)

In [80]:
import os
import dotenv
dotenv.load_dotenv(override=True)

ORACLE_DB_USER = os.environ.get("ORACLE_DB_USER", None)  #
ORACLE_DB_PASSWORD = os.environ.get("ORACLE_DB_PASSWORD", None)  #
ORACLE_DB_DSN = os.environ.get("ORACLE_DB_DSN", None)  #
ORACLE_WALLET_DIR = os.environ.get("ORACLE_WALLET_DIR", None)
ORACLE_WALLET_PASSWORD = os.environ.get("ORACLE_WALLET_PASSWORD", None)

ORACLE_DB_POOL_MIN = int(os.environ.get("ORACLE_DB_POOL_MIN", 2))
ORACLE_DB_POOL_MAX = int(os.environ.get("ORACLE_DB_POOL_MAX", 10))
ORACLE_DB_POOL_INCREMENT = int(os.environ.get("ORACLE_DB_POOL_INCREMENT", 1))

## Vector Search 테이블/인덱스 생성 예제

In [85]:
def create_table(connection, table_name):
    with connection.cursor() as cursor:
        try:
            log.info(f"Creating Table {table_name}")
            cursor.execute(
                f"""
                BEGIN
                    EXECUTE IMMEDIATE '
                        CREATE TABLE IF NOT EXISTS {table_name} (
                            id VARCHAR2(255) PRIMARY KEY,
                            document  CLOB,
                            embedding   vector
                        )
                    ';
                EXCEPTION
                    WHEN OTHERS THEN
                        IF SQLCODE != -955 THEN
                            RAISE;
                        END IF;
                END;
            """
            )


        
            connection.commit()
            log.info("Create Table initialization completed successfully.")

        except Exception as e:
            connection.rollback()
            log.exception(f"Error during database initialization: {e}")
            raise

def create_vector_index(connection, table_name):
    with connection.cursor() as cursor:
        try:
            log.info(f"Creating Index {table_name}_id_idx")
            cursor.execute(
                f"""
                BEGIN
                    EXECUTE IMMEDIATE '
                        CREATE INDEX IF NOT EXISTS {table_name}_id_idx
                        ON {table_name} (id)
                    ';
                EXCEPTION
                    WHEN OTHERS THEN
                        IF SQLCODE != -955 THEN
                            RAISE;
                        END IF;
                END;
            """
            )

            log.info(f"Creating VECTOR INDEX {table_name}_vector_ivf_idx")
            cursor.execute (
                f"""
                BEGIN
                    EXECUTE IMMEDIATE '
                        CREATE VECTOR INDEX IF NOT EXISTS {table_name}_vector_ivf_idx 
                        ON {table_name}(embedding) 
                        ORGANIZATION NEIGHBOR PARTITIONS
                        DISTANCE COSINE
                        WITH TARGET ACCURACY 95
                        PARAMETERS (TYPE IVF, NEIGHBOR PARTITIONS 30)
                    ';
                EXCEPTION
                    WHEN OTHERS THEN
                        IF SQLCODE != -955 THEN
                            RAISE;
                        END IF;
                END;
            """)
    
            connection.commit()
            log.info("Create Vector Index initialization completed successfully.")

        except Exception as e:
            connection.rollback()
            log.exception(f"Error during database initialization: {e}")
            raise


def drop_table(connection, table_name):
    with connection.cursor() as cursor:
        try:
            log.info(f"Dropping TABLE {table_name}")
            cursor.execute (
                f"""
                BEGIN
                    EXECUTE IMMEDIATE '
                        DROP TABLE IF EXISTS {table_name}
                       ';
                EXCEPTION
                    WHEN OTHERS THEN
                        IF SQLCODE != -955 THEN
                            RAISE;
                        END IF;
                END;
            """)
    
            connection.commit()
            log.info("Drop Table completed successfully.")

        except Exception as e:
            connection.rollback()
            log.exception(f"Error during database initialization: {e}")
            raise

## 데이터 로드

In [82]:
import json 
import pandas as pd
import oracledb
import array
from typing import List, Tuple
from langchain_oci import OCIGenAIEmbeddings


def load_data(connection: oracledb.Connection, 
              table_name: str, 
              data: pd.DataFrame,
              embeddings: OCIGenAIEmbeddings,
              batch_size: int = 96) -> int:
    """
    CSV 데이터를 읽어서 embedding을 생성하고 Oracle DB에 저장
    
    Args:
        connection: Oracle DB 연결 객체
        table_name: 저장할 테이블 이름
        data: pandas DataFrame (id, document 컬럼 필수)
        embeddings: OCIGenAIEmbeddings 객체
        batch_size: 배치 처리 크기 (기본값: 96)
    
    Returns:
        int: 삽입된 레코드 수
    """
    try:
        cursor = connection.cursor()
        
        # document 컬럼 확인
        if "document" not in data.columns:
            raise ValueError("데이터에 'document' 컬럼이 없습니다")
        
        # id 컬럼 확인 및 생성
        if "id" not in data.columns:
            data["id"] = [f"doc_{i}" for i in range(len(data))]
        
        # 결측치 제거
        data = data.dropna(subset=["document"])
        total_rows = len(data)
        
        print(f"총 {total_rows}개 문서 처리 시작...")
        
        # 배치로 임베딩 생성
        all_embeddings = []
        for i in range(0, total_rows, batch_size):
            batch_end = min(i + batch_size, total_rows)
            batch = data.iloc[i:batch_end]
            documents = batch["document"].tolist()
            
            # 배치 임베딩 생성
            batch_embeddings = embeddings.embed_documents(documents)
            all_embeddings.extend(batch_embeddings)
            
            print(f"  임베딩 생성: {batch_end}/{total_rows}")
        
        # DB 삽입용 데이터 준비
        rows = []
        for i, row in data.iterrows():
            # Oracle VECTOR 타입을 위해 array.array('f', embedding) 변환
            embedding_array = array.array('f', all_embeddings[len(rows)])
            rows.append((row["id"], row["document"], embedding_array))
        
        # DB에 삽입
        insert_sql = f"""
        INSERT INTO {table_name} (id, document, embedding)
        VALUES (:1, :2, :3)
        """
        
        cursor.executemany(insert_sql, rows)
        connection.commit()
        
        print(f"✓ {len(rows)}개 레코드 삽입 완료")
        
        cursor.close()
        return len(rows)
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        connection.rollback()
        raise

# auth_profile = "DEFAULT"  로 바꾸어서 사용할 것.



In [83]:
connection = oracledb.connect(
    user=ORACLE_DB_USER,
    password=ORACLE_DB_PASSWORD,
    dsn=ORACLE_DB_DSN,
    config_dir=ORACLE_WALLET_DIR,
    wallet_location=ORACLE_WALLET_DIR,
    wallet_password=ORACLE_WALLET_PASSWORD,
) 
from langchain_oci import OCIGenAIEmbeddings


model_id = "cohere.embed-v4.0"

compartment_id = "ocid1.compartment.oc1..aaaaaaaalpw23nt2lebhpddcvkangwux6igx3jxvtpbr4cnm5j62xlkxh2sa"
service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
auth_profile="DEFAULT" #### USE THIS 
auth_profile="ETEVERS"

# OCI embedding 모델 설정
embeddings = OCIGenAIEmbeddings(
    model_id=model_id,  # 또는 OCI에서 제공하는 embedding 모델 ID
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
    auth_profile=auth_profile
    )
    



In [86]:
table_names = [
    "ddl",
    "sql_qa",
    "documentation"
]

table_data= [
    "data/ddl.csv",
    "data/sql_qa.csv",
    "data/documentation.csv"
]

for i in range(len(table_names)):

    data = table_data[i]
    table_name = table_names[i]
    df = pd.read_csv(data)

    drop_table(connection,table_name)
    create_table(connection,table_name)
    
    # 데이터 로드 실행
    inserted_count = load_data(
        connection=connection,
        table_name=table_name,
        data=df,
        embeddings=embeddings,
        batch_size=1024
    )
    create_vector_index(connection,table_name)
    




2025-12-10 18:11:47,704 - __main__ - INFO - Dropping TABLE ddl
2025-12-10 18:11:48,106 - __main__ - INFO - Drop Table completed successfully.
2025-12-10 18:11:48,107 - __main__ - INFO - Creating Table ddl
2025-12-10 18:11:48,119 - __main__ - INFO - Create Table initialization completed successfully.
2025-12-10 18:11:48,243 - __main__ - INFO - Creating Index ddl_id_idx
2025-12-10 18:11:48,247 - __main__ - INFO - Creating VECTOR INDEX ddl_vector_ivf_idx


총 11개 문서 처리 시작...
  임베딩 생성: 11/11
✓ 11개 레코드 삽입 완료


2025-12-10 18:11:48,732 - __main__ - INFO - Create Vector Index initialization completed successfully.
2025-12-10 18:11:49,962 - __main__ - INFO - Dropping TABLE sql_qa
2025-12-10 18:11:52,674 - __main__ - INFO - Drop Table completed successfully.
2025-12-10 18:11:52,675 - __main__ - INFO - Creating Table sql_qa
2025-12-10 18:11:52,685 - __main__ - INFO - Create Table initialization completed successfully.


총 7707개 문서 처리 시작...
  임베딩 생성: 1024/7707
  임베딩 생성: 2048/7707
  임베딩 생성: 3072/7707
  임베딩 생성: 4096/7707
  임베딩 생성: 5120/7707
  임베딩 생성: 6144/7707
  임베딩 생성: 7168/7707
  임베딩 생성: 7707/7707


2025-12-10 18:12:50,557 - __main__ - INFO - Creating Index sql_qa_id_idx
2025-12-10 18:12:50,562 - __main__ - INFO - Creating VECTOR INDEX sql_qa_vector_ivf_idx


...

2025-12-10 18:12:56,832 - __main__ - INFO - Create Vector Index initialization completed successfully.
2025-12-10 18:12:57,108 - __main__ - INFO - Dropping TABLE documentation
2025-12-10 18:12:59,158 - __main__ - INFO - Drop Table completed successfully.
2025-12-10 18:12:59,159 - __main__ - INFO - Creating Table documentation
2025-12-10 18:12:59,171 - __main__ - INFO - Create Table initialization completed successfully.


총 1791개 문서 처리 시작...
  임베딩 생성: 1024/1791
  임베딩 생성: 1791/1791


2025-12-10 18:13:10,312 - __main__ - INFO - Creating Index documentation_id_idx
2025-12-10 18:13:10,317 - __main__ - INFO - Creating VECTOR INDEX documentation_vector_ivf_idx


✓ 1791개 레코드 삽입 완료


2025-12-10 18:13:12,361 - __main__ - INFO - Create Vector Index initialization completed successfully.


In [77]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine(
    f"oracle+oracledb://{ORACLE_DB_USER}:{ORACLE_DB_PASSWORD}@{ORACLE_DB_DSN}",
    connect_args={
        "config_dir": ORACLE_WALLET_DIR,   # tnsnames.ora / sqlnet.ora 가 있는 디렉토리
        "wallet_location": ORACLE_WALLET_DIR,
        "wallet_password": ORACLE_WALLET_PASSWORD
    }
)

df = pd.read_sql(f"SELECT * FROM {table_names[0]}", engine)
df

,id,document,embedding
0,6bc29ab9-f24b-5174-851d-bfb5d214d7d2-ddl,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN...","[-0.010115309618413448, -0.014238747768104076,..."
1,d51d6c77-3de3-5ad2-a23e-ba851316bdd0-ddl,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ...","[0.015324573963880539, 0.00350005691871047, 0...."
2,2f3abc46-57ff-5fed-b809-f343373971d3-ddl,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN...","[-0.0005660079186782241, -0.004749716725200415..."
3,90df7c1a-e4b4-5975-a2e1-713c8fa7f66f-ddl,"CREATE TABLE ""customers""\r\n(\r\n [Customer...","[-0.00621633417904377, -0.000478340225527063, ..."
4,0f111144-3373-5893-b925-68dd8a5e56bb-ddl,"CREATE TABLE ""employees""\r\n(\r\n [Employee...","[0.001335750101134181, -0.013009045273065567, ..."
5,98bb3ef3-465c-54fa-94a3-ac17d55e801e-ddl,"CREATE TABLE ""playlist_track""\r\n(\r\n [Pla...","[0.04593156278133392, -0.04113650694489479, 0...."
6,3a9c3fe7-c4b1-5398-8b66-703c0ab03902-ddl,"CREATE TABLE ""tracks""\r\n(\r\n [TrackId] IN...","[0.0036947582848370075, -0.011770911514759064,..."
7,b76ad558-edb4-57f3-b1b3-dcfb4b1605c1-ddl,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId...","[-0.0001233322109328583, -0.01512600015848875,..."
8,b305e78f-be4e-54df-a40a-6ec55439912d-ddl,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo...","[0.017854319885373116, -0.028515532612800598, ..."
9,4cbfd7f8-d4e7-5d61-8399-82e582dc39cf-ddl,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT...","[0.002867489354684949, -0.014511713758111, -0...."
